<a href="https://colab.research.google.com/github/guilhermebleidao/pdi-2022-2/blob/main/Tarefa1/Tarefa1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando arquivos e bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import zipfile
import os

!wget --no-check-certificate \
    "https://github.com/guilhermebleidao/pdi-2022-2/archive/refs/heads/main.zip" \
    -O "/tmp/pdi-2022-2.zip"

zip_ref = zipfile.ZipFile('/tmp/pdi-2022-2.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
image = plt.imread('/tmp/pdi-2022-2-main/Tarefa1/Base1/Hulk.png')
plt.imshow(image, cmap='gray')